In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
import mpmath
from iterative_refinement_utils import IR_QHD

/Users/samuelkushnir/miniconda3/envs/QHDOPT-dev/lib/python3.9/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [2]:
# Set the desired precision (number of decimal places)
mpmath.mp.dps = 50  # for example, 50 decimal places
# Set A explicitly for this example:
A = np.array([[2, 0], [0, 3]])
# Convert A to mpmath matrix
A_mpmath = mpmath.matrix(A)

# High precision x vector
x_mpmath = mpmath.matrix([0.5, 0.33333333333333333333])

# Calculate b using mpmath's dot product
b_mpmath = A_mpmath * x_mpmath


res, c = IR_QHD(A_mpmath, b_mpmath, 1e-16)

Error at 1th step: 0.10000014305114740542634876874217297881841659545898
Error at 2th step: 0.025000125169753972809161268742172978818416595458984


KeyboardInterrupt: 

Harder Problem

In [4]:
A = np.array([[2.3413, 1.858], [1.858, 0.563]])
# Convert A to mpmath matrix
A_mpmath = mpmath.matrix(A)

# High precision x vector
x_mpmath = mpmath.matrix([0.34987313872918378423, 0.534563452331234876327])
A_mpmath = A_mpmath * A_mpmath.transpose()

# Calculate b using mpmath's dot product
b_mpmath = A_mpmath * x_mpmath


res, c = IR_QHD(A_mpmath, b_mpmath, 1e-16)

Received Task from D-Wave:
2024-02-05 17:17:34
Backend QPU Time: 0.02762997
Overhead Time: 6.139835955216675

0.095146324746063333622592845886094031848068481434762
Received Task from D-Wave:
2024-02-05 17:17:54
Backend QPU Time: 0.02780357
Overhead Time: 4.323018401893311

0.038338876415363294773416776002613046843878034013572
Received Task from D-Wave:
2024-02-05 17:18:10
Backend QPU Time: 0.03158237
Overhead Time: 9.081648812098388

0.028768975563847665524706203672308336155140007436724
Received Task from D-Wave:
2024-02-05 17:18:43
Backend QPU Time: 0.02787477
Overhead Time: 27.947911200687866

0.0063007368883041022814374327888738053938570372388421
Received Task from D-Wave:
2024-02-05 17:19:11
Backend QPU Time: 0.03081677
Overhead Time: 27.20747210939453

0.0013008472048886950973654562623973295044637715687906
Received Task from D-Wave:
2024-02-05 17:19:42
Backend QPU Time: 0.02798717
Overhead Time: 31.279914927702027

0.0013183843134461790608344949776018288574300889241562
Received Ta

In [5]:
c

matrix(
[['0.34987313872918382465226321281973602935339057456857'],
 ['0.53456345233123487258488309982956639131506721926777']])